In [1]:
import os
import shutil
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib.pylab import mpl
import struct
import serial
from pathlib import Path

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False
g = 9.801538877

def move_dir(src,dest):
    if not os.path.exists(dest):
        os.makedirs(dest)
    shutil.move(src,dest)
def split_pro(data):
    split_symbol = [' ',',','，']
    for symbol in split_symbol:
        data = '#*#'.join(data.split(symbol))
    return [item for item in data.split('#*#') if item]

In [2]:
lw_skip = 0
lw_list_sort = [0,1,2]
bd_skip = 4
bd_list_sort = [2,0,1]
ga_shift = 3

gsl_point = 10
gsl_list_sort = [2,1,0]
zf_list = [-1,-1,1]
sl_list = [10,50,100,150,180]


def fun_lw_average(df,lw_skip,lw_list_sort,bd_skip,bd_list_sort,ga_shift=3):
    glw_list = []
    gbd_list = []
    alw_list = []
    abd_list = []
    aname_list = []
    fzj_list = []
    axis_name_list = ['x','y','z']
    all_ag_list = [glw_list,gbd_list,alw_list,abd_list]
    for item in all_ag_list:
        for i in range(3):
            item.append([])
    
    df.drop(df.columns[0],axis=1,inplace=True)

    for i in range(3):

        glw_mean = df.iloc[ i*4+lw_skip: (i+1)*4+lw_skip, lw_list_sort[i] ]
        alw_mean = df.iloc[ i*4+lw_skip: (i+1)*4+lw_skip, lw_list_sort[i]+ga_shift ]
        glw_list[lw_list_sort[i]].append(glw_mean.mean())
        alw_list[lw_list_sort[i]].append(alw_mean.mean())

        gbd_mean = df.iloc[ i*4+bd_skip: (i+1)*4+bd_skip, bd_list_sort[i] ]
        abd_mean = df.iloc[ i*4+bd_skip: (i+1)*4+bd_skip, bd_list_sort[i]+ga_shift ]
        gbd_list[bd_list_sort[i]].append( (gbd_mean[gbd_mean>0].mean()-gbd_mean[gbd_mean<0].mean())/2 )
        abd_list[bd_list_sort[i]].append( (abd_mean[abd_mean>0].mean()-abd_mean[abd_mean<0].mean())/2 )

    for i in range(3):
        if i==0:
            sqrt_gxyz = df.iloc[:,i]**2
            sqrt_axyz = df.iloc[:,i+ga_shift]**2
        else:
            sqrt_gxyz += df.iloc[:,i]**2
            sqrt_axyz += df.iloc[:,i+ga_shift]**2
    
    for i in range(3):
        for j in range(3):
            if i==j:
                continue
            t_name = axis_name_list[bd_list_sort[i]]
            o_name = axis_name_list[bd_list_sort[j]]
            aname = 'K{}{}'.format(o_name,t_name)
            aname_list.append(aname)
            zd_data = df.iloc[ i*4+bd_skip: (i+1)*4+bd_skip, bd_list_sort[i]+ga_shift ]
            cd_data = df.iloc[ i*4+bd_skip: (i+1)*4+bd_skip, bd_list_sort[j]+ga_shift ]
            fzj_data = (np.arcsin(cd_data/zd_data)/np.pi*180*60)[-6:].mean()
            fzj_list.append(fzj_data)

    sqrt_gxyz_mean = np.sqrt(sqrt_gxyz.mean())
    sqrt_axyz_mean = np.sqrt(sqrt_axyz.mean())
    return gbd_list,glw_list,abd_list,alw_list,sqrt_gxyz_mean,sqrt_axyz_mean,aname_list,fzj_list

# 使用示例
df = pd.read_csv('D:\\Data\\D501\\20240928_D501#01_02_03\\D501#01\\LW_26_4_s_average.txt',sep='\\s+',header=None,skiprows=1)
fun_lw_average(df,lw_skip,lw_list_sort,bd_skip,bd_list_sort)


def fun_sl_average(df,gsl_point,gsl_list_sort,sl_list,zf_list):
    all_sl_list = []
    bd_list = [[],[],[]]
    fxxd_list = [[],[],[]]
    gname_list = []
    fzj_list = []


    axis_name_list = ['x','y','z']
    for i in sl_list:
        for j in [1,-1]:
            all_sl_list.append(i*j)
    sl_point_length = len(all_sl_list)
    # print(all_sl_list)
    df.drop(df.columns[0],axis=1,inplace=True)

    for i in range(3):
        bd_data = df.iloc[ i*sl_point_length:(i+1)*sl_point_length, gsl_list_sort[i] ] * zf_list[i] / 3600 
        # if i==0:
        #     print(list(bd_data))
        data_x = list(bd_data)
        data_y = all_sl_list
        fits = np.polyfit(data_y,data_x,1)
        # erros = data_x - data_y*fits[0]-fits[1]
        erros = [ data_x[i] - data_y[i]*fits[0]-fits[1] for i in range(len(data_x))]
        bd_list[gsl_list_sort[i]].append(fits[0])
        fxxd_list[gsl_list_sort[i]].append( float((pd.DataFrame(erros).abs().max()/fits[0]*1e6/180).iloc[0]) )
    for i in range(3):
        for j in range(3):
            if i==j:
                continue
            t_name = axis_name_list[gsl_list_sort[i]]
            o_name = axis_name_list[gsl_list_sort[j]]
            gname = 'K{}{}'.format(o_name,t_name)
            gname_list.append(gname)
            zd_data = df.iloc[ i*sl_point_length:(i+1)*sl_point_length, gsl_list_sort[i] ] * zf_list[i]
            cd_data = df.iloc[ i*sl_point_length:(i+1)*sl_point_length, gsl_list_sort[j] ] * zf_list[i]
            fzj_data = (np.arcsin(cd_data/zd_data)/np.pi*180*60)[-6:].mean()
            fzj_list.append(fzj_data)

    return bd_list, fxxd_list, gname_list, fzj_list

df = pd.read_csv('D:\\Data\\D501\\20240928_D501#01_02_03\\D501#01\\SL_26_6_s_average.txt',sep='\\s+',header=None,skiprows=1)
# fun_sl_average(df,gsl_point,gsl_list_sort,sl_list,zf_list)

In [3]:
# 重新整理数据格式
# from_path = '../D501/D501#240806/'
# to_path = '../D501/20240806_4/'
from_path = '../某型无人机/20240924_加表测试/'
# to_path = '../某型无人机/20240924_加表测试/'
from_path = 'D:\\Data\\机载63\\20240925_63加表测试\\'
from_path = 'D:\\Data\\机载63\\20240926_63#01加表测试\\'
from_path = 'D:\\Data\\D501\\20240928_D501#01_02_03\\'
from_path = 'D:\\Data\\D501\\20240928_D501#01_02_03\gw\\'
from_path = 'D:\\Data\\机载63\\20240929_63#01加表测试\\'
from_path = 'D:\\Data\\D501\\20240928#D501#04_03\\'
from_path = 'D:\\Data\\D501\\20240928_D501#04_03\\'
from_path = 'D:\\Data\\D501\\20241104#D501#04_03#gwSL\\'
from_path = 'D:\\Data\\D501\\20241122#交付验证#D501#05\\'
to_path = from_path
for test_name in os.listdir(from_path):
    for fog_name in os.listdir(from_path+test_name):
        fog_path = from_path+test_name+'/'+fog_name+'/'
        to_fog_path = to_path+fog_name+'/'+test_name+'/'
        try:
            for data_name in os.listdir(fog_path):
                move_dir(fog_path+data_name,to_fog_path)
            print('移动文件夹成功: {} -> {}'.format(fog_path+data_name,to_fog_path))
        except Exception as e:
            print('移动文件夹失败: {} {}'.format(fog_path,e))

移动文件夹成功: D:\Data\D501\20241122#交付验证#D501#05\JT_-50_1/861#05_BD/861#05_BD_BD1#loc[0.0_0.0_0.0_0.0]#22271_s.txt -> D:\Data\D501\20241122#交付验证#D501#05\861#05_BD/JT_-50_1/
移动文件夹成功: D:\Data\D501\20241122#交付验证#D501#05\JT_-50_2/861#05_BD/861#05_BD_BD1#loc[0.0_90.0_0.0_0.0]#235755_s.txt -> D:\Data\D501\20241122#交付验证#D501#05\861#05_BD/JT_-50_2/
移动文件夹成功: D:\Data\D501\20241122#交付验证#D501#05\JT_26_1/861#05_BD/861#05_BD_BD1#loc[0.0_0.0_0.0_0.0]#71445_s.txt -> D:\Data\D501\20241122#交付验证#D501#05\861#05_BD/JT_26_1/
移动文件夹成功: D:\Data\D501\20241122#交付验证#D501#05\JT_26_2/861#05_BD/861#05_BD_BD1#loc[0.0_90.0_0.0_0.0]#84539_s.txt -> D:\Data\D501\20241122#交付验证#D501#05\861#05_BD/JT_26_2/
移动文件夹成功: D:\Data\D501\20241122#交付验证#D501#05\JT_70_1/861#05_BD/861#05_BD_BD1#loc[0.0_0.0_0.0_0.0]#154924_s.txt -> D:\Data\D501\20241122#交付验证#D501#05\861#05_BD/JT_70_1/
移动文件夹成功: D:\Data\D501\20241122#交付验证#D501#05\JT_70_2/861#05_BD/861#05_BD_BD1#loc[0.0_90.0_0.0_0.0]#172017_s.txt -> D:\Data\D501\20241122#交付验证#D501#05\861#05_BD/JT_

In [11]:
# 逐行扫描文件夹下的文件并取均值存放
# 适用文件格式： 给定目标文件夹/ [LW SL JT]/ [file1.txt file2.txt file3.txt]

# load_paths = 'D:\\Data\\D501\\20240922_testo\\D501#99\\'
# load_paths = 'D:\\Data\\机载63\\20240925_63加表测试\\D501#01\\'
load_paths = 'D:\\Data\\机载63\\20240926_63#01加表测试\\65#01加表测试\\'
load_paths = 'D:\\Data\\D501\\20240928_D501#01_02_03\\D501#01\\'
load_paths =  'D:\\Data\\机载63\\20240929_63#01加表测试\\65#01加表测试2\\'
load_paths = 'D:\\Data\\某型无人机\\20251017_wrj#02\\'
load_paths = 'D:\\Data\\D501\\20240928#D501#04_03\\861#2304003\\'
load_paths = 'D:\\Data\\126\\20241101_lsBD\\'
load_paths = 'D:\\Data\\126\\20241101_19wzbd\\'
load_paths = 'D:\\Data\\D501\\20241101#D501#04_03\\861#2304003\\'
load_paths = 'D:\\Data\\D501\\20240928_D501#04_03\\861#2304004\\'
# load_paths = 'D:\\Data\\D501\\20241104#D501#04_03#gwSL\\861#003_GWSLYZ\\'
load_paths = 'D:\\Data\\激光\\JG#04JFCS_20241105\\'
load_paths = 'D:\\Data\\D501\\20241122#交付验证#D501#05\\861#05_BD\\'
# 初始化全局
cal_hz = False
cal_se = True
save_title = True
axis = ['X','Y','Z']
count_begin_rule = 'BD'
count_end_rule = '#'
debug_out_list = []
std_rolls = 1
begin_axis = 0
# begin_axis = 17
# begin_axis = 17

# begin_axis:begin_axis+3 *= ↓
right3_axis_para = 3600 
# right3_axis_para = 1
# begin_axis+3:begin_axis+6 *= ↓
right36_axis_para = 1

# right3_axis_para = 0.071
# right36_axis_para = 1/312719

# 初始化标题
title_list = ['TurnTable']
for name in axis:
    title_list.append('G{}_mean'.format(name))
for name in axis:
    title_list.append('A{}_mean'.format(name))
for name in axis:
    title_list.append('G{}_temp'.format(name))
for name in axis:
    title_list.append('A{}_temp'.format(name))
for name in axis:
    title_list.append('G{}_std'.format(name))
for name in axis:
    title_list.append('A{}_std'.format(name))
title_list.append('average_length')
title_list.append('count_diff')

load_paths = Path(load_paths)
for path_name in os.listdir(load_paths):
    if not os.path.isdir(load_paths/path_name):
        continue
    save_name_hz = load_paths/'{}_hz_average.txt'.format(path_name)
    save_name_s =  load_paths/'{}_s_average.txt'.format(path_name)
    save_title_list = title_list
    if 'jt_' in path_name.lower():
        for i in axis:
            save_title_list += ['Hz_roll3_max_{}'.format(i)]
        
    if cal_hz:
        with open(save_name_hz,'w+') as f:
            f.write('\t'.join(save_title_list)+'\n')
    if cal_se:
        with open(save_name_s,'w+') as f:
            f.write('\t'.join(save_title_list)+'\n')

    try:
        for filename in sorted( os.listdir(load_paths/path_name), key= lambda itemname:int(itemname.split(count_begin_rule)[1].split(count_end_rule)[0]) ):
            # print('{} / {}'.format(path_name, filename))
            if (not cal_hz) & ('hz.txt' in filename):
                continue
            if (not cal_se) & ('s.txt' in filename):
                continue
            pd_name = load_paths/path_name/filename
            turn_table = filename.split('#')[-2]
            try:
                files = pd.read_csv(pd_name,sep='\\s+',header=None,skiprows=1, encoding='gb2312')
            except:
                print('pd打开文件失败:{}'.format(pd_name))

            diff_data = files.iloc[:,0].diff().mean()
            files.drop(files.columns[0],axis=1,inplace=True)

            files.iloc[:,begin_axis:begin_axis+3] *= right3_axis_para
            files.iloc[:,begin_axis+3:begin_axis+6] *= right36_axis_para

            if 'lw_' in path_name.lower():
                ends = 56
            elif 'sl_' in path_name.lower():
                ends = 36
            elif 'jt_' in path_name.lower():
                ends = 3600
            else:
                ends = -1
            files = files[:ends]
            
            mean_data = files.iloc[:,begin_axis:begin_axis+3*4].mean()
            stds_data = files.iloc[:,begin_axis:begin_axis+3*2].rolling(std_rolls).mean().std()
            len_data = len(files)
            save_data = '{}\t{}\t{}\t{}\t{}'.format(
                turn_table,
                '\t'.join(['{:.6f}'.format(i) for i in mean_data]),
                '\t'.join(['{:.6f}'.format(i) for i in stds_data]),
                len_data,diff_data
            )

            if 'jt_' in path_name.lower():
                try:
                    pd_name = load_paths/path_name/(filename.replace('s.txt','hz.txt'))
                    files = pd.read_csv(pd_name,sep='\\s+',header=None,skiprows=1, encoding='gb2312')
                    files = files[-3600*200:]
                    rolls_data = files.rolling(4).mean()
                    for i in range(3):
                        save_data+='\t{:.6f}'.format(rolls_data[i+1].max())
                    for i in range(3):
                        save_data+='\t{}'.format(rolls_data[i+1].idxmax())
                except Exception as e:
                    print('追加静态毫秒值失败:{} {}'.format(filename,e))
            
            if cal_hz:
                with open(save_name_hz,'a+') as f:
                    f.write(save_data+'\n')
            if cal_se:
                with open(save_name_s,'a+') as f:
                    f.write(save_data+'\n')

    except Exception as e:
        debug_out_list.append('{} {} {} '.format(load_paths, path_name, e))
        

In [14]:
# 计算各点零位与标度初步处理结果
# 默认无标度因数，初步处理完成后生成标度参数文件
# 切换标志位使用参数文件

load_paths = 'D:\\Data\\D501\\20240928_D501#01_02_03\\D501#02\\'
load_paths = 'D:\\Data\\D501\\20240928_D501#04_03\\861#2304004\\'
# load_paths = 'D:\\Data\\D501\\20241104#D501#04_03#gwSL\\861#003_GWSLYZ\\'
load_paths = 'D:\\Data\\激光\\JG#04JFCS_20241105\\'

load_paths = 'D:\\Data\\D501\\20241122#交付验证#D501#05\\861#05_BD\\'
load_paths = Path(load_paths)

# 初始化参数
us_bd_file = False
load_rule_list = ['SL','JT','LW']
lw_skip = 0
lw_list_sort = [0,2,1]
bd_skip = 4
bd_list_sort = [1,0,2]
gsl_point = 10
gsl_list_sort = [1,2,0]
# gsl_list_sort = [1,2,0]
zf_list = [1,-1,1]
rounds = 8
fzj_name_list = ['Kyx','Kzx','Kxy','Kzy','Kxz','Kyz']

# 初始化保存列表
gbd_list = []
gfzj_list = []
alw_list = []
abd_list = []
abd_list_z = []
abd_list_f = []
for i in range(3):
    alw_list.append([])
for i in range(3):
    abd_list_z.append([])
for i in range(3):
    abd_list_f.append([])
    

title_list = ['LW_file_name']
for i in ['G','A']:
    for j in ['_BD','_LW']:
        for k in ['x','y','z']:
            title_list.append(i+k+j)
title_list.append('sqrt_Gxyz')
title_list.append('sqrt_Axyz')
for i in fzj_name_list:
    title_list.append('a'+i)
print(' '.join(title_list))
lw_save_name = 'alldata_LW.txt'
with open(load_paths/lw_save_name,'w+') as f:
    f.write('\t'.join(title_list)+'\n')


title_list = ['SL_file_name']
for i in ['x','y','z']:
    title_list.append('BD_'+i)
for i in ['x','y','z']:
    title_list.append('FXXD_'+i)
for i in fzj_name_list:
    title_list.append('g'+i)
print(' '.join(title_list))
sl_save_name = 'alldata_SL.txt'
with open(load_paths/sl_save_name,'w+') as f:
    f.write('\t'.join(title_list)+'\n')


for filename in os.listdir(load_paths):
    if os.path.isdir(load_paths/filename):
        continue
    if True not in [item.lower() in filename.lower() for item in load_rule_list]:
        continue
    if 'alldata' in filename.lower():
        continue
    # print(filename)
    df = pd.read_csv(load_paths/filename, sep='\\s+', header=None, skiprows=1)

    if 'lw' in filename.lower():
        # break
        gbd_list,glw_list,abd_list,alw_list,sqrt_gxyz_mean,sqrt_axyz_mean,aname_list,fzj_list = fun_lw_average(df,lw_skip,lw_list_sort,bd_skip,bd_list_sort)
        all_bdlw_list = [gbd_list,glw_list,abd_list,alw_list]
        save_data = filename.replace('.txt','').replace('_average','')
        for item in all_bdlw_list:
            for xyz in item:
                save_data += '\t{}'.format( round(xyz[0],rounds) )
        save_data += '\t{}'.format( round(sqrt_gxyz_mean,rounds) )
        save_data += '\t{}'.format( round(sqrt_axyz_mean,rounds) )
        for item in fzj_name_list:
            item_data = fzj_list[ aname_list.index(item) ]
            save_data+='\t{}'.format( round(item_data,rounds) )

        with open(load_paths/lw_save_name,'a+') as f: 
            f.write(save_data+'\n')

        continue
    elif 'sl' in filename.lower():
        bd_list, fxxd_list, gname_list, fzj_list = fun_sl_average(df,gsl_point,gsl_list_sort,sl_list,zf_list)
        all_bdsl_list = [bd_list,fxxd_list]
        save_data = filename.replace('.txt','').replace('_average','')
        for item in all_bdsl_list:
            for xyz in item:
                save_data+='\t{}'.format( round(xyz[0],rounds) )
        for item in fzj_name_list:
            item_data = fzj_list[ gname_list.index(item) ]
            save_data+='\t{}'.format( round(item_data,rounds) )
        with open(load_paths/sl_save_name,'a+') as f: 
            f.write(save_data+'\n')


        continue
    elif 'jt' in filename.lower():
        continue
    else:
        debug_out_list.append('未知文件:{} 当前适用:{}'.format(filename,' '.join(load_rule_list)))

LW_file_name Gx_BD Gy_BD Gz_BD Gx_LW Gy_LW Gz_LW Ax_BD Ay_BD Az_BD Ax_LW Ay_LW Az_LW sqrt_Gxyz sqrt_Axyz aKyx aKzx aKxy aKzy aKxz aKyz
SL_file_name BD_x BD_y BD_z FXXD_x FXXD_y FXXD_z gKyx gKzx gKxy gKzy gKxz gKyz


In [75]:
aname_list

['Kyz', 'Kxz', 'Kzy', 'Kxy', 'Kzx', 'Kyx']

In [76]:
fzj_list

[[],
 [],
 [],
 -1.625947554961506,
 0.5221039894232566,
 1.5576749234907727,
 -21.310270263635154,
 -0.783892252854921,
 21.354475650766613]

In [43]:
df.drop(df.columns[0],axis=1,inplace=True)
for i in range(3):
    glw_mean = df.iloc[ i*4+lw_skip: (i+1)*4+lw_skip, lw_list_sort[i] ]
    print(glw_mean)

0    1.555650
1   -1.744714
2   -1.780650
3    1.698879
Name: 1, dtype: float64
4    1.653236
5   -1.645329
6   -1.708393
7    1.746707
Name: 2, dtype: float64
8    -11.428714
9    -11.429229
10    11.457129
11    11.440286
Name: 3, dtype: float64


In [49]:
load_paths

WindowsPath('D:/Data/D501/20240928_D501#04_03/861#2304004')

In [46]:
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.555650,11.483357,9.576064,0.007464,-0.005535,9.801959,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,0.079518,0.065710,0.099139,0.000133,0.000130,0.000109,56,0.0
1,-1.744714,-11.431864,9.585321,0.001547,-0.006966,9.802035,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,0.084029,0.085517,0.072163,0.000136,0.000080,0.000067,56,0.0
2,-1.780650,11.451150,-9.583779,-0.005631,0.006191,-9.800754,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,0.099493,0.065358,0.088953,0.000135,0.000114,0.000119,56,0.0
3,1.698879,-11.388214,-9.605636,0.000089,0.004680,-9.800684,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,0.106074,0.065024,0.081320,0.000129,0.000094,0.000139,56,0.0
4,-11.550793,1.653236,9.633214,0.003777,-0.003284,9.801967,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,0.066935,0.080592,0.128898,0.000091,0.000140,0.000062,56,0.0
5,11.373943,-1.645329,9.639193,0.005080,-0.009200,9.802039,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,0.067757,0.092182,0.089776,0.000119,0.000081,0.000111,56,0.0
6,-11.507979,-1.708393,-9.617014,-0.003514,0.002653,-9.800621,-47.761964,-47.761964,-47.761964,-45.000000,-45.000000,-45.000000,0.078014,0.083774,0.106788,0.000079,0.000104,0.000105,56,0.0
7,11.373043,1.746707,-9.575871,-0.002076,0.008280,-9.800649,-47.000000,-47.000000,-47.000000,-45.000000,-45.000000,-45.000000,0.060032,0.082860,0.075937,0.000149,0.000096,0.000094,56,0.0
8,1.547100,9.618621,-11.428714,-0.042593,9.801008,0.004740,-47.000000,-47.000000,-47.000000,-45.022321,-45.022321,-45.022321,0.100934,0.068076,0.075027,0.000188,0.000069,0.000134,56,0.0
9,-1.726457,-9.609879,-11.429229,0.044456,-9.802014,-0.006033,-47.000000,-47.000000,-47.000000,-46.000000,-46.000000,-46.000000,0.104889,0.088852,0.068857,0.000144,0.000091,0.000136,56,0.0


In [302]:
# 使用alldata绘图

load_paths = 'D:\\Data\\D501\\20240928_D501#01_02_03\\D501#01\\'
load_paths = Path(load_paths)
save_path = load_paths/'figure'
if not save_path:
    os.mkdir(save_path)

In [303]:
save_path

WindowsPath('D:/Data/D501/20240928_D501#01_02_03/D501#01/figure')